<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/05_Gestores_reportados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import pandas as pd

# Ruta del archivo Excel que ya tiene la TABLA_UNIFICADA

ruta_tabla_unificada = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_GESTOR.xlsx'
ruta_llamadas_parquet ='/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/INFORME_GESTORESREPOR.xlsx'

tabla_unificada = pd.read_excel(ruta_tabla_unificada, sheet_name='TABLA_UNIFICADAGEST')

llamadas = pd.read_parquet(ruta_llamadas_parquet)

# Inspección rápida de valores únicos antes de convertir
print("Valores únicos antes de conversión:")
print(tabla_unificada['%_CUMPLIMIENTO'].unique())

# Función robusta para limpiar y convertir
def limpiar_y_convertir(valor):
    try:
        if pd.isna(valor):
            return None
        valor = str(valor).replace('%', '').replace(',', '.').strip()
        return float(valor)
    except:
        print(f"❌ Error al convertir: {valor}")
        return None
tabla_unificada['%_CUMPLIMIENTO'] = tabla_unificada['%_CUMPLIMIENTO'].apply(limpiar_y_convertir)


Valores únicos antes de conversión:
[100.     0.    75.    66.67  50.    83.33  71.43  80.    87.5   85.71
  33.33  92.86  90.91  88.89  90.    77.78  60.    94.12  72.73  62.5
  86.67  57.14  55.56  93.33  82.35  40.    93.75  91.67  88.24  81.82
  20.    25.    70.    14.29]


In [ ]:
# 1. Definir la relación VARIABLE PADRE → SUBVARIABLES
# --------------------------
mapeo_subvars = {
    'P1': ['P11', 'P12', 'P13', 'P14', 'P15'],
    'P2': ['P21', 'P22', 'P23'],
    'P3': ['P31', 'P32', 'P33'],
    'S1': ['S11', 'S12', 'S13'],
    'S3': ['S31', 'S32', 'S33', 'S34'],
    'S4': ['S41', 'S42', 'S43', 'S44', 'S45', 'S46'],
    'S5M': ['S51', 'S52'],
    'C1M': ['C11', 'C12', 'C13'],
    'C2M': ['C21', 'C22'],
    'T1M': ['T11', 'T12', 'T13', 'T14'],
    'T2M': ['T21', 'T22'],
    'T3': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37']
}


# --------------------------
# 2. Crear columna con subvariables True para cada fila (llamada)
# --------------------------
def get_true_subvars(row, variable_padre):
    subvars = mapeo_subvars.get(variable_padre, [])
    true_vars = [var for var in subvars if var in row and row[var] == True]
    return true_vars


# --------------------------
# 3. Agregamos columna auxiliar con la variable padre en cada llamada
# --------------------------
detalles = []

for variable_padre, subvars in mapeo_subvars.items():
    # Asegurarse que las subvars existen en el DataFrame
    columnas_presentes = [col for col in subvars if col in llamadas.columns]
    if not columnas_presentes:
        continue

    temp = llamadas[['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME'] + columnas_presentes].copy()
    temp['VARIABLE'] = variable_padre
    temp['SUBVARIABLES_TRUE'] = temp.apply(lambda row: get_true_subvars(row, variable_padre), axis=1)

    detalles.append(temp[['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VARIABLE', 'SUBVARIABLES_TRUE']])

# Unimos todos los detalles
df_subvars_true = pd.concat(detalles)

# Agrupar para cada bloque y juntar las subvariables que han salido True
df_subvars_true = df_subvars_true.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VARIABLE'])[
    'SUBVARIABLES_TRUE'] \
    .apply(lambda x: ', '.join(sorted(set(sum(x, []))))) \
    .reset_index()

# --------------------------
# 4. Unirlo con la tabla unificada final
# --------------------------
tabla_final = tabla_unificada.merge(df_subvars_true,
                                    on=['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VARIABLE'],
                                    how='left')


# --------------------------
# 6. Añadir descripción de la variable
# --------------------------
mapeo_descripciones = {
    'P1': 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA',
    'P2': 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD',
    'P3': 'ESCUCHA COMPRENSIVA Y EMPÁTICA',
    'P4': 'TRATO CORDIAL AL CLIENTE',
    'P5': 'RESPONSABILIDAD CORPORATIVA',
    'P6': 'DESPEDIDA CORPORATIVA',
    'S1': 'INFORMACIÓN MOTIVO DE LA LLAMADA',
    'S2': 'SITUACIÓN ACTUAL DEL DEUDOR',
    'S3': 'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES',
    'S4': 'DIRECCIÓN',
    'S5M': 'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS',
    'C1M': 'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO',
    'C2M': 'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE',
    'T1M': 'USO EFECTIVO DEL SCRIPT',
    'T2M': 'CAPACIDAD DE EXPRESIÓN',
    'T3': 'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN'
}

# Crear nueva columna con la descripción
tabla_final['DESC_VARIABLE'] = tabla_final['VARIABLE'].map(mapeo_descripciones)

# Reordenar columnas para que DESC_VARIABLE esté al lado de VARIABLE
columnas = list(tabla_final.columns)
if 'VARIABLE' in columnas and 'DESC_VARIABLE' in columnas:
    columnas.remove('DESC_VARIABLE')  # quitamos primero
    idx_variable = columnas.index('VARIABLE')  # localizamos dónde está VARIABLE
    columnas.insert(idx_variable + 1, 'DESC_VARIABLE')  # insertamos justo después

    tabla_final = tabla_final[columnas]  # aplicamos el reordenamiento

# --------------------------

# Diccionario de descripciones de subvariables

desc_subvars = {
    'P11': 'No se presenta "en nombre de la entidad" de la deuda (Monoproducto)',
    'P12': 'No dice nombre y apellido gestor',
    'P13': 'No dice nombre agencia',
    'P14': 'No informa de la grabación de la llamada',
    'P15': 'No se presenta "en nombre de CaixaBank" y especifica la entidad de cada deuda (Multiproducto)',

    'P21': 'No verifica DNI',
    'P22': 'No pregunta por el cliente deudor',
    'P23': 'Incumplimiento compliance grabación llamadas (numeración tarjeta)',

    'P31': 'No demuestra interés por la situación del cliente',
    'P32': 'No tiene en cuenta la información trasladada por el cliente para conseguir un acuerdo',
    'P33': 'Adopta una actitud inapropiada de cara al cliente (sarcástica, condescendiente, de reproche…)',

    'S11': 'No especifica claramente el motivo de la llamada',
    'S12': 'No especifica el importe de la deuda',
    'S13': 'No especifica la antigüedad de la deuda',

    'S31': 'No se identifica la capacidad de pago',
    'S32': 'El gestor no hace referencia al siguiente vencimiento cuando cierran un acuerdo',
    'S33': 'No sondea, presupone la situación del cliente',
    'S34': 'Intenta terminar la llamada rápido, quitándosela de encima',

    'S41': 'No se percibe que el gestor domine la conversación, más bien ésta dirigida por el cliente',
    'S42': 'No consigue linealidad y concreción',
    'S43': 'Actúa a "la defensiva"',
    'S44': 'Ante un cliente complejo/insatisfecho genera confrontación',
    'S45': 'Alarga o precipita de manera innecesaria el cierre de conversación',
    'S46': 'No dirige la secuencia de llamada manteniendo el enfoque',

    'S51': 'No concreta fecha e importe en caso de acuerdo de pago',
    'S52': 'No concreta fecha de seguimiento en caso de no haber acuerdo de pago',

    'C11': 'No ofrece tarjeta de débito CaixaBank',
    'C12': 'No ofrece tarjeta de crédito otra entidad',
    'C13': 'No ofrece tarjeta de familiar/amigo',

    'C21': 'No ofrecen las alternativas de pago disponibles (aparte de TPV)',
    'C22': 'Enfoque incorrecto con exceso de acuerdos parciales',

    'T11': 'No hace uso efectivo del script (en todos los casos salvo protocolo negociación con oficina)',
    'T12': 'No sigue el procedimiento "negociación deuda con oficina" como objeción del cliente',
    'T13': 'No sigue con los pasos habituales para el recobro (la información facilitada por el cliente es insuficiente)',
    'T14': 'No le indica al cliente que se toma nota de lo tramitado con la oficina (la información facilitada por el cliente es suficiente)',

    'T21': 'No busca el compromiso de pago enunciando beneficios/consecuencias',
    'T22': 'No es exigente acordando una fecha',

    'T31': 'No adecúa su lenguaje y su forma de expresarse al grado de compresión del cliente',
    'T32': 'No se expresa con claridad y profesionalidad',
    'T33': 'No interactúa en la comunicación con el cliente',
    'T34': 'Utiliza un tono de voz inapropiado (mecánico/agresivo/pasivo…)',
    'T35': 'No muestra seguridad en la comunicación',
    'T36': 'No afronta las objeciones con argumentos adecuados, correctos y completos',
    'T37': 'No aplica una solución adaptada al perfil del cliente'
}

# Función para mapear las descripciones
def mapear_descripciones(subvars_str):
    if pd.isna(subvars_str) or not isinstance(subvars_str, str):
        return ''
    subvars = [s.strip() for s in subvars_str.split(',')]
    descripciones = [desc_subvars.get(s, s) for s in subvars]
    return ', '.join(descripciones)

# --------------------------
# Eliminar columnas duplicadas (por si acaso)
tabla_final = tabla_final.loc[:, ~tabla_final.columns.duplicated()]

# Añadir columna con descripciones de subvariables
tabla_final['DESC_SUBVARIABLES'] = tabla_final['SUBVARIABLES_TRUE'].apply(mapear_descripciones)

# Reordenar para colocarla justo al lado de SUBVARIABLES_TRUE
cols = tabla_final.columns.tolist()
if 'DESC_SUBVARIABLES' in cols:
    cols.remove('DESC_SUBVARIABLES')

if 'SUBVARIABLES_TRUE' in cols:
    idx = cols.index('SUBVARIABLES_TRUE')
    cols = cols[:idx+1] + ['DESC_SUBVARIABLES'] + cols[idx+1:]
else:
    cols.append('DESC_SUBVARIABLES')

tabla_final = tabla_final[cols]

tabla_final = tabla_final.sort_values(by='%_CUMPLIMIENTO', ascending=False)

In [ ]:
# 5. Guardar todo en un nuevo Excel
# --------------------------
with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
    tabla_final.to_excel(writer, sheet_name='TABLA_UNIFICADA_CON_SUBVARS', index=False)

print(f"✅ Archivo generado en: {ruta_salida}")

✅ Archivo generado en: /content/drive/MyDrive/COPS/OUTPUT/INFORME_GESTORESREPOR.xlsx
